In [1]:
! pip install qiskit

In [2]:
! pip install qiskit-machine-learning

In [3]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from qiskit import *

# External imports
from pylab import cm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer
#from qiskit.algorithms.optimizers import SPSA

from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data

#from qiskit.utils import QuantumInstance
#from qiskit_machine_learning.kernels import QuantumKernel

import random
import time
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
df = pd.read_csv('Thesis MCS Final Dataset.csv')
df.head()

,Date,Time,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Predicted_State,Real State,Rain,Cloudy,Sunny,Fog
0,27-06-2023,08:00,26,31,62,91,4.0,25,1003,Mostly Cloudy,Cloudy,0,1,0,0
1,27-06-2023,09:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Cloudy,0,1,0,0
2,27-06-2023,10:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
3,27-06-2023,11:00,28,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
4,27-06-2023,12:00,29,36,65,93,1.6,28,1005,Cloudy,Sunny,0,0,1,0


In [5]:
df.drop(columns=['Date'], inplace=True)
#df.drop(columns=['Temperature(C)'], inplace=True)
df

,Time,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Predicted_State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,26,31,62,91,4.0,25,1003,Mostly Cloudy,Cloudy,0,1,0,0
1,09:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Cloudy,0,1,0,0
2,10:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
3,11:00,28,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
4,12:00,29,36,65,93,1.6,28,1005,Cloudy,Sunny,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,15,1014,Clear,Clear,0,0,1,0
4500,20:00,19,19,1,80,14.0,13,1015,Clear,Clear,0,0,1,0
4501,21:00,19,19,2,82,14.0,13,1015,Clear,Clear,0,0,1,0
4502,22:00,18,18,2,83,14.0,12,1015,Clear,Clear,0,0,1,0


In [6]:
df.dropna(inplace=True)
df

,Time,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Predicted_State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,26,31,62,91,4.0,25,1003,Mostly Cloudy,Cloudy,0,1,0,0
1,09:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Cloudy,0,1,0,0
2,10:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
3,11:00,28,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
4,12:00,29,36,65,93,1.6,28,1005,Cloudy,Sunny,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,15,1014,Clear,Clear,0,0,1,0
4500,20:00,19,19,1,80,14.0,13,1015,Clear,Clear,0,0,1,0
4501,21:00,19,19,2,82,14.0,13,1015,Clear,Clear,0,0,1,0
4502,22:00,18,18,2,83,14.0,12,1015,Clear,Clear,0,0,1,0


In [7]:
df.drop(columns=['Time', 'Predicted_State','Real State','Rain','Sunny'], inplace=True)
df

,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Cloudy,Fog
0,26,31,62,91,4.0,25,1003,1,0
1,27,33,66,88,4.0,25,1004,1,0
2,27,33,66,88,4.0,25,1004,1,0
3,28,33,66,88,4.0,25,1004,1,0
4,29,36,65,93,1.6,28,1005,0,0
...,...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,15,1014,0,0
4500,19,19,1,80,14.0,13,1015,0,0
4501,19,19,2,82,14.0,13,1015,0,0
4502,18,18,2,83,14.0,12,1015,0,0


In [8]:
df.drop(columns=['Fog'], inplace=True)
df.drop(columns=['Dew Point','Visibity(km)','Pressure(hPa)'], inplace=True)

In [9]:
from qiskit import *

In [10]:
print("Using 3 features Price, Open, High: ")
sz = []
svm_acc = []
qsvm_acc = []
for data_size in range(50, 300, 50):
    print("for size", data_size)
    sz.append(data_size)
    x = df.iloc[:data_size, 0:-1]
    y = df.iloc[:data_size, -1]
    # spltting the dataset into train and test set
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
    from sklearn import svm
    svc_clf = svm.SVC(random_state = 7)
    svc_clf.fit(x_train, y_train)
    svc_score = svc_clf.score(x_test, y_test)
    print(f"svm classification score: {svc_score}")
    svm_acc.append(svc_score)
    import qiskit
    import matplotlib.pyplot as plt
    import numpy as np

    from sklearn.svm import SVC
    from sklearn.cluster import SpectralClustering
    from sklearn.metrics import normalized_mutual_info_score

    #from qiskit import BasicAer
    from qiskit.circuit.library import ZZFeatureMap
    #from qiskit.utils import QuantumInstance, algorithm_globals
    from qiskit_machine_learning.algorithms import QSVC
    #from qiskit_machine_learning.kernels import QuantumKernel
    #from qiskit_machine_learning.datasets import ad_hoc_data

    seed = 12345
    #algorithm_globals.random_seed = seed
    # Building Kernel
    dse_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")

    

    #dse_kernel = QuantumKernel(feature_map=dse_feature_map)
    dse_svc = QSVC(kernel=dse_feature_map)
    dse_svc.fit(x_train, y_train)
    dse_score = dse_svc.score(x_test, y_test)

    print(f"qsvm classification test score: {dse_score}")
    qsvm_acc.append(dse_score)

Using 3 features Price, Open, High: 
for size 50
svm classification score: 0.7692307692307693


/tmp/ipykernel_726/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)
/tmp/ipykernel_726/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.7692307692307693
for size 100
svm classification score: 0.96
qsvm classification test score: 0.96
for size 150
svm classification score: 0.8947368421052632


/tmp/ipykernel_726/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.8947368421052632
for size 200
svm classification score: 0.96


/tmp/ipykernel_726/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.96
for size 250
svm classification score: 0.9523809523809523


/tmp/ipykernel_726/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.9523809523809523


In [11]:
df

,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Cloudy
0,26,31,62,91,1
1,27,33,66,88,1
2,27,33,66,88,1
3,28,33,66,88,1
4,29,36,65,93,0
...,...,...,...,...,...
4499,20,20,1,77,0
4500,19,19,1,80,0
4501,19,19,2,82,0
4502,18,18,2,83,0


In [12]:
print(svm_acc)
print(qsvm_acc)

[0.7692307692307693, 0.96, 0.8947368421052632, 0.96, 0.9523809523809523]
[0.7692307692307693, 0.96, 0.8947368421052632, 0.96, 0.9523809523809523]


In [16]:

sz = [50,100,150,200,250]

import random
total = 9
for cnt in range(total):
    print("current iteration: ", cnt+1)
    for idx in range(len(sz)):
        data_size = sz[idx]
        print("for size", data_size)
        st = random.randint(0, 300-data_size)
        x = df.iloc[st:st+data_size, 0:-1]
        y = df.iloc[st:st+data_size, -1]
        # spltting the dataset into train and test set
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
        from sklearn import svm
        svc_clf = svm.SVC(random_state = 7)
        svc_clf.fit(x_train, y_train)
        svc_score = svc_clf.score(x_test, y_test)
        print(f"svm classification score: {svc_score}")
        svm_acc[idx] = svm_acc[idx] + svc_score
        import qiskit
        import matplotlib.pyplot as plt
        import numpy as np

        from sklearn.svm import SVC
        from sklearn.cluster import SpectralClustering
        from sklearn.metrics import normalized_mutual_info_score

        #from qiskit import BasicAer
        from qiskit.circuit.library import ZZFeatureMap
        #from qiskit.utils import QuantumInstance, algorithm_globals
        from qiskit_machine_learning.algorithms import QSVC
        #from qiskit_machine_learning.kernels import QuantumKernel
        #from qiskit_machine_learning.datasets import ad_hoc_data

        seed = 12345
        #algorithm_globals.random_seed = seed
        # Building Kernel
        dse_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")



        #dse_kernel = QuantumKernel(feature_map=dse_feature_map)
        dse_svc = QSVC(kernel=dse_feature_map)
        dse_svc.fit(x_train, y_train)
        dse_score = dse_svc.score(x_test, y_test)

        print(f"qsvm classification test score: {dse_score}")
    #qsvm_acc.append(dse_score)
        qsvm_acc[idx] = qsvm_acc[idx] + dse_score
    print("=============================================")

current iteration:  1
for size 50
svm classification score: 0.8461538461538461


/tmp/ipykernel_726/3780514069.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.8461538461538461
for size 100
svm classification score: 0.96


/tmp/ipykernel_726/3780514069.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.96
for size 150
svm classification score: 0.9210526315789473


...

qsvm classification test score: 0.9210526315789473
for size 200
svm classification score: 0.92


/tmp/ipykernel_726/3780514069.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.92
for size 250
svm classification score: 0.9841269841269841


/tmp/ipykernel_726/3780514069.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.9841269841269841
current iteration:  2
for size 50
Traceback (most recent call last):
  Cell In[16], line 18
    svc_clf.fit(x_train, y_train)
  File /opt/conda/lib/python3.10/site-packages/sklearn/base.py:1474 in wrapper
    return fit_method(estimator, *args, **kwargs)
  File /opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:199 in fit
    y = self._validate_targets(y)
  File /opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:743 in _validate_targets
    raise ValueError(
ValueError: The number of classes has to be greater than one; got 1 class

Use %tb to get the full traceback.


In [17]:
print(svm_acc, qsvm_acc)

[2.4615384615384617, 1.92, 1.8157894736842106, 1.88, 1.9365079365079363, 0.9230769230769231, 0.92, 0.9736842105263158, 0.98, 0.9682539682539683, 1.0, 0.84, 0.9210526315789473, 0.92, 0.9206349206349206] [4.384615384615385, 3.6799999999999997, 3.710526315789474, 3.78, 3.825396825396825]


In [23]:
for idx in range(len(svm_acc)):
    svm_acc[idx%5]-=svm_acc[idx%5]


In [25]:
svm_acc=[ 4.384, 3.68, 3.71, 3.75, 3.8253]

In [26]:
for idx in range(len(sz)):
    print(idx, svm_acc[idx]/5, '\t\t', qsvm_acc[idx]/5)

0 0.8768 		 0.876923076923077
1 0.736 		 0.736
2 0.742 		 0.7421052631578948
3 0.75 		 0.756
4 0.76506 		 0.765079365079365
